Importing required packages

In [17]:
import os
import cv2
import gc
from skimage import color, data, restoration
import cv2
import numpy as np
from skimage.restoration import estimate_sigma
from skimage.filters import median
import config
import imutils
import warnings
warnings.filterwarnings('ignore')


adding function that process the input images

In [18]:
def weiner_noise_reduction(img):
    # data.astronaut()
    img = color.rgb2gray(img)
    from scipy.signal import convolve2d
    psf = np.ones((5, 5)) / 25
    img = convolve2d(img, psf, 'same')
    img += 0.1 * img.std() * np.random.standard_normal(img.shape)
    deconvolved_img = restoration.wiener(img, psf, 1100)

    return deconvolved_img



def estimate_noise(img):
    # img = cv2.imread(image_path)
    return estimate_sigma(img, multichannel=True, average_sigmas=True)


def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    enoise = estimate_noise(image)
    noise_free_image = weiner_noise_reduction(image)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    fingerprint = gray - noise_free_image
    fingerprint = fingerprint / 255
    filtered_img = median(fingerprint, selem=None, out=None, mask=None, shift_x=False,
                          shift_y=False, mode='nearest', cval=0.0, behavior='rank')
    colored = cv2.cvtColor(filtered_img, cv2.COLOR_GRAY2BGR)
    # print('-----------------')
    # cv2.imshow('filtered_image', filtered_img)
    # colored = cv2.cvtColor(filtered_img, cv2.COLOR_GRAY2BGR)
    # print(colored)
    # cv2.imshow('colored', colored)
    return colored


function to process casia one image dataset and store them into numpy matrix with file name <b>'dataset/np_casia_one_forged.npy'<b>

In [19]:

def prepare_casia_one_dataset():
    casia_one_au_arr = []
    casia_one_forged_arr = []

    # np.save('data.npy', num_arr) # save
    for image in os.listdir(CASIA_ONE_AUTHENTIC_PATH):
        imagepath = os.path.join(CASIA_ONE_AUTHENTIC_PATH, image)
        cv_image = cv2.imread(imagepath)
        print(str(image) + 'processing...')
        h, w = cv_image.shape[:2]
        if h != 256 and w != 384:
            continue
            # cv_image = imutils.resize(cv_image, width=384, height=256)
        if h == 256 and w == 384:
            processed_image = preprocess_image(cv_image)
            casia_one_au_arr.append(np.array(processed_image))
        else:
            print('Dimention mismatch')

    np_casia_one_au = np.array(casia_one_au_arr)
    np.save('dataset/np_casia_one_au.npy', np_casia_one_au)  # save
    print('CASIA1 Authentic Data Processed..')
    gc.collect()

    for image in os.listdir(CASIA_ONE_FORGED_PATH):
        imagepath = os.path.join(CASIA_ONE_FORGED_PATH, image)
        cv_image = cv2.imread(imagepath)
        print(str(image) + 'processing...')
        h, w = cv_image.shape[:2]
        if h != 256 and w != 384:
            continue
            # cv_image = imutils.resize(cv_image, width=384, height=256)
        if h == 256 and w == 384:
            processed_image = preprocess_image(cv_image)
            casia_one_forged_arr.append(np.array(processed_image))
        else:
            print('Dimention mismatch')

    np_casia_one_forged = np.array(casia_one_forged_arr)
    np.save('dataset/np_casia_one_forged.npy', np_casia_one_forged)  # save
    print('CASIA1 Forged Data Processed..')
    gc.collect()


CASIA 1 database contains <b>800 authentic</b> and <b>921 forged</b> images.

The size s <b>384X256</b> pixels.


In [20]:

CASIA_ONE_AUTHENTIC_PATH = 'casia-dataset/CASIA1/Au/'
CASIA_ONE_FORGED_PATH = 'casia-dataset/CASIA1/Sp/'

Checking... is there already process numpy array exist or not. 
If not exists then creating new one.

In [21]:
filename = os.path.join('dataset', 'np_casia_one_forged.npy')
if not os.path.exists(filename):
    print('Processing Casia I dataset...')
    prepare_casia_one_dataset()
else:
    print(filename + ' already processed...')

dataset/np_casia_one_forged.npy already processed...


The <b>CASIA 2</b> database contains more than <b>7400 authentic</b> and <b>5000 forged images</b>.
The images are in either JPEG, TIFF, or BMP format.


In [22]:

CASIA_TWO_AUTHENTIC_PATH = 'casia-dataset/CASIA2/Au/'
CASIA_TWO_FORGED_PATH = 'casia-dataset/CASIA2/Tp/'

function to process casio two dataset

In [23]:
def prepare_casia_two_dataset():
    casia_two_au_arr = []
    casia_two_forged_arr = []

    # np.save('data.npy', num_arr) # save
    for image in os.listdir(CASIA_TWO_AUTHENTIC_PATH):
        imagepath = os.path.join(CASIA_TWO_AUTHENTIC_PATH, image)
        cv_image = cv2.imread(imagepath)
        try:
            print(str(image) + 'processing...')
            h, w = cv_image.shape[:2]
            if h != 256 and w != 384:
                continue
                # cv_image = imutils.resize(cv_image, width=384, height=256)
            if h == 256 and w == 384:
                processed_image = preprocess_image(cv_image)
                casia_two_au_arr.append(np.array(processed_image))
            else:
                print('Dimention mismatch')
        except Exception as err:
            print(err)

    np_casia_two_au = np.array(casia_two_au_arr)
    np.save('dataset/np_casia_two_au.npy', np_casia_two_au)  # save
    print('CASIA2 Authentic Data Processed..')
    gc.collect()

    for image in os.listdir(CASIA_TWO_FORGED_PATH):
        imagepath = os.path.join(CASIA_TWO_FORGED_PATH, image)
        cv_image = cv2.imread(imagepath)
        try:
            print(str(image) + 'processing...')
            h, w = cv_image.shape[:2]
            if h != 256 and w != 384:
                continue
                # cv_image = imutils.resize(cv_image, width=384, height=256)
            if h == 256 and w == 384:
                processed_image = preprocess_image(cv_image)
                casia_two_forged_arr.append(np.array(processed_image))
            else:
                print('Dimention mismatch')
        except Exception as err:
            print(err)


    np_casia_two_forged = np.array(casia_two_forged_arr)
    np.save('dataset/np_casia_two_forged.npy', np_casia_two_forged)  # save
    print('CASIA2 Forged Data Processed..')
    gc.collect()


Checking... is there already process numpy array exist or not. 
If not exists then creating new one.

In [24]:
filename = os.path.join('dataset', 'np_casia_two_forged.npy')
if not os.path.exists(filename):
    print('Processing Casia II dataset...')
    prepare_casia_two_dataset()
else:
    print(filename + ' already processed...')


dataset/np_casia_two_forged.npy already processed...


#### Training the keras classifier

In [25]:
import keras
from keras import Model, Sequential, optimizers, applications
from keras.applications import ResNet50
from keras.layers import GlobalAveragePooling2D, Dropout, Dense, Flatten
from keras_applications import resnet50
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import config

In [26]:

XAuthenticate = list(np.load(config.np_casia_one_au_path))
yAuthenticate = list(np.zeros(len(XAuthenticate), dtype=np.uint8))
XForged = list(np.load(config.np_casia_one_forged_path))
yForged = list(np.ones(len(XForged), dtype=np.uint8))

X = np.array(XAuthenticate + XForged)
y = np.array(yAuthenticate + yForged, dtype=np.int8)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True)


build the VGG16 network

In [27]:
img_height = 256
img_width = 384

# build the VGG16 network
model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

build a classifier model to put on top of the convolutional model

In [28]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

add the model on top of the convolutional base

In [29]:
# model.add(top_model) this throws error alternative is below

new_model = Sequential() #new model
for layer in model.layers:
    new_model.add(layer)

new_model.add(top_model) # now this works

set the first 25 layers (up to the last conv block) to non-trainable (weights will not be updated)

LOCK THE TOP CONV LAYERS

In [30]:
for layer in new_model.layers[:15]:
    layer.trainable = False

print('Model loaded.')

print(new_model.summary())

Model loaded.
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 256, 384, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 384, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 192, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 192, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 192, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 96, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 64, 

In [31]:
# model_aug.load_weights('99 % accurate model.h5')
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
new_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])


In [ ]:

new_model.fit(x_train, y_train,
          epochs=10,
          batch_size=5)

new_model.evaluate(x_test, y_test, verbose=0)

y_pred = new_model.predict_classes(x_test)

plt.clf()

Epoch 1/10
 425/1177 [=========>....................] - ETA: 26:42 - loss: 7.2560 - accuracy: 0.5365

In [ ]:
print(confusion_matrix(y_test, y_pred))

In [ ]:
# new_model.save(filepath='casia2_model.h5')
# new_model.save_weights('bottleneck_fc_model.h5')